# Visualization Demo

In [ ]:
import napari
%gui qt5

In [ ]:
from brainlit.utils.session import NeuroglancerSession
from brainlit.utils.swc import *
from brainlit.viz.visualize import *
import numpy as np
from skimage import io
from pathlib import Path

## Loading Data

In [ ]:
data_dir = str(Path().resolve().parents[2] / "tests" / "upload" / "test_precomputed_serial")
swc_dir = str(Path().resolve().parents[2] / "tests" / "upload_segments")
dest_dir = str(Path().resolve().parents[0] / "utils" / "upload")
dest_dir_segments = str(Path().resolve().parents[0] / "utils" / "upload_segments")
themip = 0
sid = 2
vid = 300

In [ ]:
# Create a neuroglancersession
ngl_sess = NeuroglancerSession(
    url="file://"+dest_dir, 
    url_segments="file://"+dest_dir_segments,
    mip=themip
)

In [ ]:
img, bbbox, vox = ngl_sess.pull_voxel(sid, vid, 1, 1, 1)

In [ ]:
skel = ngl_sess.cv_segments.skeleton.get(sid)
vertex = skel.vertices[vid]/ngl_sess.cv_segments.scales[themip]["resolution"]
print(vertex)

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.squeeze(np.array(img)))
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='vertex')

## 2D Visualization

In [ ]:
plot_image_2d(img[:,100,:])

In [ ]:
fig1, axes = plot_image_2d(img[50,:,:])
axes.set_title('First Dimension')

fig2, axes = plot_image_2d(img[:,50,:])
axes.set_title('Second Dimension')
    
fig3, axes = plot_image_2d(img[:,:,50])
axes.set_title('Third Dimension')

## Max intensity projection

In [ ]:
fig, axis = plot_image_mip(img)
print(type(fig))

## Histogram Plot of Image Intensities

In [ ]:
fig = plot_image_hist(img)

## Visualizing all successors or predecessors of a given vertex up to a given depth

In [ ]:
s3_path = "file://"+dest_dir_segments
seg_id = 2
df = read_s3(s3_path, seg_id, mip=themip)
df.head()

In [ ]:
G = df_to_graph(df)
print('Number of nodes:', len(G.nodes))
print('Number of edges:', len(G.edges))
print('\n')
print('Sample 1 coordinates (x,y,z)')
print(G.nodes[1]['x'],G.nodes[1]['y'],G.nodes[1]['z'])

paths = graph_to_paths(G=G)
print(f"The graph was decomposed into {len(paths)} paths")

In [ ]:
viewer = napari.Viewer(ndisplay=3)
viewer.add_image(np.squeeze(np.array(img)))
# viewer.add_points(data=np.concatenate(paths, axis=0)[1024:], edge_width=2, edge_color='white', name='all_points')
viewer.add_shapes(data=paths, shape_type='path', edge_color='white', edge_width=3, name='skeleton')
viewer.add_points(data=np.array(vertex), edge_width=5, edge_color='green', name='vertex')

In [ ]:
v_id = 44  # the row index/number of the data frame
depth = 30  # the depth up to which the graph must be constructed

G_bfs=get_bfs_subgraph(G, v_id, depth, df=df)  # perform Breadth first search to obtain a graph of interest
paths_bfs = graph_to_paths(G=G_bfs[0])  # obtain all the paths for visualization purposes

In [ ]:
x = df.iloc[v_id]['x']
y = df.iloc[v_id]['y']
z = df.iloc[v_id]['z']
v_pos = np.array([x,y,z])

# display vertex
viewer = napari.Viewer(ndisplay=3)
viewer.add_points(data=v_pos, edge_width=20, edge_color='orange', name='vertex')

# display all interest nodes around vertex
viewer.add_points(data=np.concatenate(paths_bfs), edge_color='red', edge_width=2, name='subpoints')
viewer.add_shapes(data=paths_bfs, shape_type='path', edge_color='red', edge_width=3, name='sub-skeleton')